In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
from sklearn.preprocessing import OneHotEncoder


# 设置随机种子
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 在代码的开头设置随机种子
set_seed(42)

# 读取数据
data = pd.read_csv('final_50_all.csv')

data['hour'] = pd.to_datetime(data['hour'])
data['year'] = data['hour'].dt.year
data['month'] = data['hour'].dt.month
data['day'] = data['hour'].dt.day
data['hour_of_day'] = data['hour'].dt.hour
data['weekday'] = data['hour'].dt.weekday

# 添加高峰/非高峰阶段特征
def classify_peak_hour(hour):
    if 12 <= hour <= 18:  # 高峰期
        return 'peak'
    elif 0 <= hour <= 6:  # 非高峰期
        return 'off_peak'
    else:  # 其他阶段
        return 'other'

data['peak_stage'] = data['hour_of_day'].apply(classify_peak_hour)

# 数值特征
num_features = ['temperature_2m (°C)', 'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_100m (km/h)']

# 类别特征
cat_features = ['peak_stage']

# 周期性编码
data['hour_sin'] = np.sin(2 * np.pi * data['hour_of_day'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour_of_day'] / 24)
data['weekday_sin'] = np.sin(2 * np.pi * data['weekday'] / 7)
data['weekday_cos'] = np.cos(2 * np.pi * data['weekday'] / 7)
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

# 数值特征缩放
scaler = MinMaxScaler()
data[num_features] = scaler.fit_transform(data[num_features])

# 对类别变量进行 One-Hot 编码
encoder = OneHotEncoder(sparse_output=False)
encoded_cat = encoder.fit_transform(data[cat_features])
encoded_cat_columns = encoder.get_feature_names_out(cat_features)

# 拼接数值特征、周期性特征和 One-Hot 编码特征
encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoded_cat_columns, index=data.index)
data = pd.concat([data, encoded_cat_df], axis=1)

# 删除原始类别特征和其他无关列
data.drop(['month','day','hour', 'hour_of_day', 'weekday', 'peak_stage'], axis=1, inplace=True)

# 提取最终特征和目标
features = num_features + ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos','month_sin','month_cos'] + list(encoded_cat_columns)
target = 'ride_count'

X = data[features].values
y = data[target].values

# 数据标准化
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

def create_sequences(data, target, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(target[i + sequence_length])
    return np.array(X), np.array(y)

# 设置序列长度
sequence_length = 24  # 例如使用过去24小时的数据预测
X, y = create_sequences(X, y, sequence_length)

# 使用 KFold 进行交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 初始化存储结果的列表
test_losses = []
maes = []
rmses = []

# 交叉验证过程
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}")

    # 分割数据
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # 转换为张量
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

    # 数据加载器
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32)

    # GRU 模型定义
    class GRUModel(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
            super(GRUModel, self).__init__()
            self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, output_size)
            self.dropout = nn.Dropout(dropout)
        
        def forward(self, x):
            out, _ = self.gru(x)
            out = self.dropout(out[:, -1, :])
            out = self.fc(out)
            return out

    # 模型实例化
    input_size = len(features)
    hidden_size = 64
    output_size = 1
    model = GRUModel(input_size, hidden_size, output_size)

    # 损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # 训练与验证
    patience = 10
    best_val_loss = float('inf')
    early_stop_counter = 0
    epochs = 100

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch).squeeze()
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # 验证阶段
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                y_pred = model(X_batch).squeeze()
                loss = criterion(y_pred, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # 早停机制
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), f'best_model_fold{fold+1}.pth')  # 保存最佳模型
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

    # 测试集评估
    model.load_state_dict(torch.load(f'best_model_fold{fold+1}.pth'))
    model.eval()
    val_loss = 0
    y_preds = []
    y_trues = []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch).squeeze()
            y_preds.extend(y_pred.squeeze().tolist())
            y_trues.extend(y_batch.tolist())
            loss = criterion(y_pred.squeeze(), y_batch)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    mae = mean_absolute_error(y_trues, y_preds)
    rmse = mean_squared_error(y_trues, y_preds, squared=False)

    test_losses.append(val_loss)
    maes.append(mae)
    rmses.append(rmse)

    print(f"Fold {fold+1} - Val Loss: {val_loss:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# 输出所有折的平均性能
print(f"\nAverage Val Loss: {np.mean(test_losses):.4f}")
print(f"Average MAE: {np.mean(maes):.4f}")
print(f"Average RMSE: {np.mean(rmses):.4f}")


Fold 1
Epoch 1/100, Train Loss: 238.5849, Val Loss: 167.6773
Epoch 2/100, Train Loss: 99.2960, Val Loss: 79.8614
Epoch 3/100, Train Loss: 85.5305, Val Loss: 78.3905
Epoch 4/100, Train Loss: 83.4606, Val Loss: 76.9107
Epoch 5/100, Train Loss: 80.3371, Val Loss: 72.0842
Epoch 6/100, Train Loss: 78.5382, Val Loss: 71.9998
Epoch 7/100, Train Loss: 77.4543, Val Loss: 69.7158
Epoch 8/100, Train Loss: 75.4503, Val Loss: 67.1794
Epoch 9/100, Train Loss: 73.8098, Val Loss: 67.7690
Epoch 10/100, Train Loss: 72.9300, Val Loss: 69.9570
Epoch 11/100, Train Loss: 70.9610, Val Loss: 65.7837
Epoch 12/100, Train Loss: 70.8052, Val Loss: 65.6619
Epoch 13/100, Train Loss: 70.5837, Val Loss: 64.0628
Epoch 14/100, Train Loss: 69.3234, Val Loss: 63.8275
Epoch 15/100, Train Loss: 69.3369, Val Loss: 63.1267
Epoch 16/100, Train Loss: 68.5907, Val Loss: 63.4795
Epoch 17/100, Train Loss: 68.1822, Val Loss: 63.5800
Epoch 18/100, Train Loss: 67.7878, Val Loss: 64.7241
Epoch 19/100, Train Loss: 67.5823, Val Loss: 

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/352729829.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mode

Epoch 1/100, Train Loss: 236.2535, Val Loss: 171.6677
Epoch 2/100, Train Loss: 97.8270, Val Loss: 88.0535
Epoch 3/100, Train Loss: 84.2961, Val Loss: 86.0159
Epoch 4/100, Train Loss: 80.7985, Val Loss: 82.2638
Epoch 5/100, Train Loss: 78.2640, Val Loss: 78.1619
Epoch 6/100, Train Loss: 76.5851, Val Loss: 78.2583
Epoch 7/100, Train Loss: 74.7276, Val Loss: 75.1599
Epoch 8/100, Train Loss: 73.6462, Val Loss: 73.9953
Epoch 9/100, Train Loss: 72.3652, Val Loss: 72.2450
Epoch 10/100, Train Loss: 70.6062, Val Loss: 74.8741
Epoch 11/100, Train Loss: 70.3780, Val Loss: 70.2810
Epoch 12/100, Train Loss: 69.5254, Val Loss: 69.6830
Epoch 13/100, Train Loss: 69.1109, Val Loss: 70.6583
Epoch 14/100, Train Loss: 67.9313, Val Loss: 68.8390
Epoch 15/100, Train Loss: 67.9949, Val Loss: 68.8831
Epoch 16/100, Train Loss: 67.4006, Val Loss: 67.3868
Epoch 17/100, Train Loss: 67.6873, Val Loss: 66.9477
Epoch 18/100, Train Loss: 66.7356, Val Loss: 68.5112
Epoch 19/100, Train Loss: 66.4584, Val Loss: 66.2402


/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/352729829.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mode

Epoch 1/100, Train Loss: 240.0919, Val Loss: 189.1884
Epoch 2/100, Train Loss: 106.5155, Val Loss: 85.7453
Epoch 3/100, Train Loss: 86.7974, Val Loss: 84.1972
Epoch 4/100, Train Loss: 84.6563, Val Loss: 80.1379
Epoch 5/100, Train Loss: 81.5635, Val Loss: 78.8280
Epoch 6/100, Train Loss: 78.2135, Val Loss: 73.4788
Epoch 7/100, Train Loss: 76.8179, Val Loss: 70.4169
Epoch 8/100, Train Loss: 73.9543, Val Loss: 68.7368
Epoch 9/100, Train Loss: 73.2510, Val Loss: 69.6117
Epoch 10/100, Train Loss: 72.3912, Val Loss: 66.6169
Epoch 11/100, Train Loss: 71.6804, Val Loss: 66.3152
Epoch 12/100, Train Loss: 70.5909, Val Loss: 66.6915
Epoch 13/100, Train Loss: 70.6170, Val Loss: 65.3961
Epoch 14/100, Train Loss: 69.9507, Val Loss: 65.2287
Epoch 15/100, Train Loss: 69.7593, Val Loss: 65.0993
Epoch 16/100, Train Loss: 69.3596, Val Loss: 64.5629
Epoch 17/100, Train Loss: 68.6835, Val Loss: 63.5620
Epoch 18/100, Train Loss: 68.9037, Val Loss: 64.0500
Epoch 19/100, Train Loss: 67.6604, Val Loss: 63.8701

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/352729829.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mode

Epoch 1/100, Train Loss: 240.9355, Val Loss: 195.3978
Epoch 2/100, Train Loss: 114.7001, Val Loss: 89.2005
Epoch 3/100, Train Loss: 86.0340, Val Loss: 85.0330
Epoch 4/100, Train Loss: 81.3293, Val Loss: 80.0155
Epoch 5/100, Train Loss: 77.4717, Val Loss: 78.5081
Epoch 6/100, Train Loss: 73.8417, Val Loss: 74.7879
Epoch 7/100, Train Loss: 72.6364, Val Loss: 72.6958
Epoch 8/100, Train Loss: 71.1165, Val Loss: 71.0989
Epoch 9/100, Train Loss: 70.3596, Val Loss: 72.0215
Epoch 10/100, Train Loss: 70.2720, Val Loss: 71.7126
Epoch 11/100, Train Loss: 69.3609, Val Loss: 72.6599
Epoch 12/100, Train Loss: 69.4858, Val Loss: 71.5013
Epoch 13/100, Train Loss: 68.7936, Val Loss: 69.6382
Epoch 14/100, Train Loss: 68.2497, Val Loss: 71.0044
Epoch 15/100, Train Loss: 68.4355, Val Loss: 69.1215
Epoch 16/100, Train Loss: 67.7364, Val Loss: 69.3924
Epoch 17/100, Train Loss: 67.4141, Val Loss: 68.3854
Epoch 18/100, Train Loss: 66.9650, Val Loss: 69.5758
Epoch 19/100, Train Loss: 66.0097, Val Loss: 70.5170

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/352729829.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mode

Epoch 1/100, Train Loss: 241.1488, Val Loss: 191.3271
Epoch 2/100, Train Loss: 105.9162, Val Loss: 81.1070
Epoch 3/100, Train Loss: 86.3200, Val Loss: 79.7606
Epoch 4/100, Train Loss: 83.8691, Val Loss: 76.9762
Epoch 5/100, Train Loss: 82.0145, Val Loss: 74.0574
Epoch 6/100, Train Loss: 79.0698, Val Loss: 75.5826
Epoch 7/100, Train Loss: 78.6376, Val Loss: 70.9244
Epoch 8/100, Train Loss: 76.5990, Val Loss: 70.6153
Epoch 9/100, Train Loss: 76.2666, Val Loss: 69.8504
Epoch 10/100, Train Loss: 74.5204, Val Loss: 70.6590
Epoch 11/100, Train Loss: 73.8764, Val Loss: 67.5713
Epoch 12/100, Train Loss: 73.8219, Val Loss: 67.6799
Epoch 13/100, Train Loss: 72.1440, Val Loss: 65.3155
Epoch 14/100, Train Loss: 72.1795, Val Loss: 69.8509
Epoch 15/100, Train Loss: 70.9702, Val Loss: 65.1814
Epoch 16/100, Train Loss: 69.7369, Val Loss: 64.1642
Epoch 17/100, Train Loss: 69.6872, Val Loss: 63.8206
Epoch 18/100, Train Loss: 69.6996, Val Loss: 64.7282
Epoch 19/100, Train Loss: 69.1017, Val Loss: 63.3036

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/352729829.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mode

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
from sklearn.preprocessing import OneHotEncoder


# 设置随机种子
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 在代码的开头设置随机种子
set_seed(42)

# 读取数据
# 加载数据
data = pd.read_csv('final_50_all.csv')

# 特征工程
data['hour'] = pd.to_datetime(data['hour'])
data['year'] = data['hour'].dt.year
data['month'] = data['hour'].dt.month
data['day'] = data['hour'].dt.day
data['hour_of_day'] = data['hour'].dt.hour
data['weekday'] = data['hour'].dt.weekday

# 添加高峰/非高峰阶段特征
def classify_peak_hour(hour):
    if 12 <= hour <= 18:  # 高峰期
        return 'peak'
    elif 0 <= hour <= 6:  # 非高峰期
        return 'off_peak'
    else:  # 其他阶段
        return 'other'

data['peak_stage'] = data['hour_of_day'].apply(classify_peak_hour)

# 数值特征
num_features = ['temperature_2m (°C)', 'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_100m (km/h)']

# 类别特征
cat_features = ['peak_stage']

# 周期性编码
data['hour_sin'] = np.sin(2 * np.pi * data['hour_of_day'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour_of_day'] / 24)
data['weekday_sin'] = np.sin(2 * np.pi * data['weekday'] / 7)
data['weekday_cos'] = np.cos(2 * np.pi * data['weekday'] / 7)
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

# 数值特征缩放
scaler = MinMaxScaler()
data[num_features] = scaler.fit_transform(data[num_features])

# 对类别变量进行 One-Hot 编码
encoder = OneHotEncoder(sparse_output=False)
encoded_cat = encoder.fit_transform(data[cat_features])
encoded_cat_columns = encoder.get_feature_names_out(cat_features)

# 拼接数值特征、周期性特征和 One-Hot 编码特征
encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoded_cat_columns, index=data.index)
data = pd.concat([data, encoded_cat_df], axis=1)

# 删除原始类别特征和其他无关列
data.drop(['month','day','hour', 'hour_of_day', 'weekday', 'peak_stage'], axis=1, inplace=True)

# 提取最终特征和目标
features = num_features + ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos','month_sin','month_cos'] + list(encoded_cat_columns)
target = 'ride_count'

X = data[features].values
y = data[target].values

def create_sequences(data, target, sequence_length):
    """
    将时间序列数据转换为带有输入序列和目标值的形式
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(target[i + sequence_length])
    return np.array(X), np.array(y)

# 设置序列长度
sequence_length = 24  # 例如使用过去24小时的数据预测
X, y = create_sequences(X, y, sequence_length)

# K折交叉验证（5折）
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0

train_losses = []
val_losses = []
maes = []
rmses = []

for train_index, val_index in kf.split(X):
    fold += 1
    print(f"Fold {fold}/{5}")
    
    # 划分训练集和验证集
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # 转换为张量
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

    # 数据加载器
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32)
    
    # 定义GRU模型
    class GRUModel(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, dropout, cnn_channels, kernel_size):
            super(GRUModel, self).__init__()
            # 添加 CNN 层
            self.conv1d = nn.Conv1d(in_channels=input_size, out_channels=cnn_channels, kernel_size=kernel_size, padding=kernel_size // 2)
            self.relu = nn.ReLU()

            # GRU 层
            self.gru = nn.GRU(cnn_channels, hidden_size, batch_first=True)
            
            # 全连接层
            self.fc = nn.Linear(hidden_size, output_size)
            self.dropout = nn.Dropout(dropout)
        
        def forward(self, x):
            # CNN 层
            x = x.permute(0, 2, 1)  # 为 Conv1d 调整形状
            x = self.conv1d(x)  # 通过 1D 卷积层
            x = self.relu(x)    # 激活函数
            x = x.permute(0, 2, 1)  # 调整形状回 [batch_size, seq_length, cnn_channels]

            # GRU 层
            out, _ = self.gru(x)  # [batch_size, seq_length, hidden_size]

            # Dropout 和全连接层
            out = self.dropout(out[:, -1, :])  # 取最后一个时间步的输出
            out = self.fc(out)  # 全连接层
            return out

    # 模型参数
    input_size = len(features)  # 输入特征数
    hidden_size = 64           # 隐藏层大小
    output_size = 1            # 输出大小（预测目标变量）
    dropout = 0.2              # Dropout 概率
    cnn_channels = 32         # CNN 输出通道数
    kernel_size = 3            # CNN 卷积核大小

    # 实例化模型
    model = GRUModel(input_size, hidden_size, output_size, dropout=dropout, cnn_channels=cnn_channels, kernel_size=kernel_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # 训练与验证
    patience = 10
    best_val_loss = float('inf')
    early_stop_counter = 0
    epochs = 100

    # 训练循环
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)  # 前向传播
            loss = criterion(y_pred.squeeze(), y_batch)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            train_loss += loss.item()
        
        train_loss /= len(train_loader)

        # 验证阶段
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                y_pred = model(X_batch).squeeze()
                loss = criterion(y_pred, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # 早停机制
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), f'best_model_fold{fold}.pth')  # 保存最佳模型
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

    # 保存训练和验证损失
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # 测试集评估
    model.load_state_dict(torch.load(f'best_model_fold{fold}.pth'))
    model.eval()
    y_preds = []
    y_trues = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch).squeeze()
            y_preds.extend(y_pred.squeeze().tolist())
            y_trues.extend(y_batch.tolist())
            loss = criterion(y_pred.squeeze(), y_batch)

    mae = mean_absolute_error(y_trues, y_preds)
    rmse = mean_squared_error(y_trues, y_preds, squared=False)
    maes.append(mae)
    rmses.append(rmse)

    print(f"Fold {fold} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# 计算平均损失
avg_train_loss = np.mean(train_losses)
avg_val_loss = np.mean(val_losses)
avg_mae = np.mean(maes)
avg_rmse = np.mean(rmses)

print(f"Average Train Loss: {avg_train_loss:.4f}")
print(f"Average Val Loss: {avg_val_loss:.4f}")
print(f"Average MAE: {avg_mae:.4f}")
print(f"Average RMSE: {avg_rmse:.4f}")

Fold 1/5
Epoch 1/100, Train Loss: 204.7894, Val Loss: 91.4384
Epoch 2/100, Train Loss: 86.5906, Val Loss: 75.3692
Epoch 3/100, Train Loss: 78.6388, Val Loss: 70.6554
Epoch 4/100, Train Loss: 75.7024, Val Loss: 69.3971
Epoch 5/100, Train Loss: 74.1042, Val Loss: 67.5920
Epoch 6/100, Train Loss: 72.2015, Val Loss: 66.5796
Epoch 7/100, Train Loss: 71.2468, Val Loss: 66.0431
Epoch 8/100, Train Loss: 70.3596, Val Loss: 65.6783
Epoch 9/100, Train Loss: 68.8201, Val Loss: 65.7459
Epoch 10/100, Train Loss: 67.5955, Val Loss: 63.5563
Epoch 11/100, Train Loss: 66.8915, Val Loss: 63.6323
Epoch 12/100, Train Loss: 66.1365, Val Loss: 63.8210
Epoch 13/100, Train Loss: 65.1613, Val Loss: 62.3998
Epoch 14/100, Train Loss: 64.1464, Val Loss: 64.3167
Epoch 15/100, Train Loss: 63.5120, Val Loss: 61.1420
Epoch 16/100, Train Loss: 61.5598, Val Loss: 61.9842
Epoch 17/100, Train Loss: 61.8041, Val Loss: 59.0331
Epoch 18/100, Train Loss: 60.0238, Val Loss: 59.1893
Epoch 19/100, Train Loss: 59.1416, Val Loss: 

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3528582541.py:217: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mod

Fold 1 - MAE: 4.0878, RMSE: 5.7313
Fold 2/5
Epoch 1/100, Train Loss: 196.3634, Val Loss: 96.7410
Epoch 2/100, Train Loss: 84.6992, Val Loss: 81.9518
Epoch 3/100, Train Loss: 77.3031, Val Loss: 77.7919
Epoch 4/100, Train Loss: 74.7931, Val Loss: 75.8219
Epoch 5/100, Train Loss: 73.2945, Val Loss: 72.9487
Epoch 6/100, Train Loss: 72.0626, Val Loss: 73.8516
Epoch 7/100, Train Loss: 70.1716, Val Loss: 73.2821
Epoch 8/100, Train Loss: 69.2141, Val Loss: 71.6583
Epoch 9/100, Train Loss: 68.6689, Val Loss: 69.1757
Epoch 10/100, Train Loss: 67.0730, Val Loss: 70.4849
Epoch 11/100, Train Loss: 67.0268, Val Loss: 66.5309
Epoch 12/100, Train Loss: 66.4148, Val Loss: 66.5430
Epoch 13/100, Train Loss: 64.7198, Val Loss: 66.1097
Epoch 14/100, Train Loss: 64.3875, Val Loss: 66.4697
Epoch 15/100, Train Loss: 63.3571, Val Loss: 67.1587
Epoch 16/100, Train Loss: 62.8991, Val Loss: 62.6807
Epoch 17/100, Train Loss: 61.8368, Val Loss: 66.8300
Epoch 18/100, Train Loss: 61.1846, Val Loss: 61.2248
Epoch 19/1

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3528582541.py:217: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mod

Fold 2 - MAE: 4.1770, RMSE: 5.8885
Fold 3/5
Epoch 1/100, Train Loss: 207.7837, Val Loss: 94.7767
Epoch 2/100, Train Loss: 86.3047, Val Loss: 76.9876
Epoch 3/100, Train Loss: 78.8609, Val Loss: 71.8423
Epoch 4/100, Train Loss: 75.8070, Val Loss: 71.9794
Epoch 5/100, Train Loss: 73.8469, Val Loss: 68.5942
Epoch 6/100, Train Loss: 72.2570, Val Loss: 69.1808
Epoch 7/100, Train Loss: 71.0765, Val Loss: 67.9911
Epoch 8/100, Train Loss: 69.9253, Val Loss: 69.7383
Epoch 9/100, Train Loss: 68.9174, Val Loss: 67.6309
Epoch 10/100, Train Loss: 67.9670, Val Loss: 63.6482
Epoch 11/100, Train Loss: 66.7664, Val Loss: 66.0495
Epoch 12/100, Train Loss: 66.4812, Val Loss: 61.9838
Epoch 13/100, Train Loss: 66.1432, Val Loss: 60.8014
Epoch 14/100, Train Loss: 64.9798, Val Loss: 61.4828
Epoch 15/100, Train Loss: 63.4626, Val Loss: 61.3054
Epoch 16/100, Train Loss: 62.5527, Val Loss: 59.6127
Epoch 17/100, Train Loss: 62.0003, Val Loss: 60.5464
Epoch 18/100, Train Loss: 61.2827, Val Loss: 56.8100
Epoch 19/1

...

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3528582541.py:217: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mod

Fold 4 - MAE: 4.1896, RMSE: 5.8763
Fold 5/5
Epoch 1/100, Train Loss: 216.3920, Val Loss: 96.3464
Epoch 2/100, Train Loss: 88.6241, Val Loss: 79.5497
Epoch 3/100, Train Loss: 81.2194, Val Loss: 72.9459
Epoch 4/100, Train Loss: 76.6254, Val Loss: 71.3966
Epoch 5/100, Train Loss: 75.3828, Val Loss: 69.0854
Epoch 6/100, Train Loss: 73.2902, Val Loss: 67.8295
Epoch 7/100, Train Loss: 71.8468, Val Loss: 67.5555
Epoch 8/100, Train Loss: 70.8848, Val Loss: 67.1865
Epoch 9/100, Train Loss: 70.2717, Val Loss: 68.0453
Epoch 10/100, Train Loss: 69.2481, Val Loss: 72.8556
Epoch 11/100, Train Loss: 67.8391, Val Loss: 68.0382
Epoch 12/100, Train Loss: 67.2931, Val Loss: 63.1370
Epoch 13/100, Train Loss: 66.9067, Val Loss: 62.6548
Epoch 14/100, Train Loss: 66.0724, Val Loss: 62.7241
Epoch 15/100, Train Loss: 64.0543, Val Loss: 61.3897
Epoch 16/100, Train Loss: 63.9224, Val Loss: 61.7057
Epoch 17/100, Train Loss: 62.5332, Val Loss: 61.9301
Epoch 18/100, Train Loss: 62.4479, Val Loss: 60.8921
Epoch 19/1

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3528582541.py:217: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_mod

Fold 5 - MAE: 4.0981, RMSE: 5.7914
Average Train Loss: 27.7470
Average Val Loss: 35.3316
Average MAE: 4.1581
Average RMSE: 5.8243


/Users/lvsuyi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from torch.nn import HuberLoss  
import pandas as pd

data = pd.read_csv('pivot_50_peak.csv')

# 数据预处理
# 提取特征列和目标列
numerical_features = ['temperature_2m (°C)', 'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_100m (km/h)']
categorical_features = ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos','peak_stage_off_peak','peak_stage_other','peak_stage_peak']
stations = [col for col in data.columns if col.isdigit()] 
# 构造输入X和目标y
X = data[categorical_features + numerical_features].values
y = data[stations].values

# 定义时间窗口
time_window = 24

# 构造时间窗口的数据集
def create_time_window(X, y, time_window):
    X_window, y_window = [], []
    for i in range(len(X) - time_window):
        X_window.append(X[i:i+time_window])
        y_window.append(y[i+time_window])  # 预测当前时间步的目标
    return np.array(X_window), np.array(y_window)

X_window, y_window = create_time_window(X, y, time_window)

# 使用 KFold 进行交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 初始化存储结果的列表
maes = []
rmses = []

# 交叉验证过程
for fold, (train_index, val_index) in enumerate(kf.split(X_window)):
    print(f"\nFold {fold + 1}")

    # 分割数据
    X_train, X_val = X_window[train_index], X_window[val_index]
    y_train, y_val = y_window[train_index], y_window[val_index]

    # 转换为Tensor
    X_train, X_val = map(torch.tensor, (X_train, X_val))
    y_train, y_val = map(torch.tensor, (y_train, y_val))

    # 数据集类
    class TimeSeriesDataset(Dataset):
        def __init__(self, X, y):
            self.X = X
            self.y = y

        def __len__(self):
            return len(self.X)

        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

    # 数据加载器
    train_dataset = TimeSeriesDataset(X_train, y_train)
    val_dataset = TimeSeriesDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # GRU 模型定义
    class GRUModel(nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
            super(GRUModel, self).__init__()
            self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            out, _ = self.gru(x)
            out = self.fc(out[:, -1, :])  # 只取最后一个时间步的输出
            return out

    # 模型初始化
    input_dim = X_train.shape[2]
    hidden_dim = 64
    output_dim = y_train.shape[1]
    num_layers = 2
    dropout = 0.3
    model = GRUModel(input_dim, hidden_dim, output_dim, num_layers, dropout)

    # 使用 Huber Loss 替代 MSELoss，并设置 delta 参数
    from torch.nn import HuberLoss
    delta = 1.0  # 可以根据数据分布或验证集表现调整该值
    criterion = HuberLoss(delta=delta)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # 训练模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_val_loss = float('inf')
    patience = 5
    trigger_times = 0

    for epoch in range(100):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.float().to(device), y_batch.float().to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)  # 使用 Huber Loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.float().to(device), y_batch.float().to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)  # 使用 Huber Loss
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # 早停机制
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trigger_times = 0
            torch.save(model.state_dict(), f'best_gru_model_fold{fold+1}.pth')  # 保存最佳模型
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping!")
                break

    # 加载最佳模型
    model.load_state_dict(torch.load(f'best_gru_model_fold{fold+1}.pth'))

    # 测试模型
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.float().to(device)
            outputs = model(X_batch)
            y_pred.append(outputs.cpu().numpy())
            y_true.append(y_batch.numpy())

    y_pred = np.vstack(y_pred)
    y_true = np.vstack(y_true)

    # 评估指标
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    maes.append(mae)
    rmses.append(rmse)

    print(f"Fold {fold+1} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# 输出所有折的平均性能
print(f"\nAverage MAE: {np.mean(maes):.4f}")
print(f"Average RMSE: {np.mean(rmses):.4f}")




Fold 1
Epoch 1, Train Loss: 0.1846, Val Loss: 0.1804
Epoch 2, Train Loss: 0.1791, Val Loss: 0.1791
Epoch 3, Train Loss: 0.1783, Val Loss: 0.1788
Epoch 4, Train Loss: 0.1777, Val Loss: 0.1786
Epoch 5, Train Loss: 0.1771, Val Loss: 0.1783
Epoch 6, Train Loss: 0.1766, Val Loss: 0.1778
Epoch 7, Train Loss: 0.1760, Val Loss: 0.1770
Epoch 8, Train Loss: 0.1754, Val Loss: 0.1767
Epoch 9, Train Loss: 0.1747, Val Loss: 0.1764
Epoch 10, Train Loss: 0.1745, Val Loss: 0.1761
Epoch 11, Train Loss: 0.1740, Val Loss: 0.1754
Epoch 12, Train Loss: 0.1735, Val Loss: 0.1753
Epoch 13, Train Loss: 0.1730, Val Loss: 0.1748
Epoch 14, Train Loss: 0.1724, Val Loss: 0.1745
Epoch 15, Train Loss: 0.1722, Val Loss: 0.1747
Epoch 16, Train Loss: 0.1718, Val Loss: 0.1739
Epoch 17, Train Loss: 0.1711, Val Loss: 0.1743
Epoch 18, Train Loss: 0.1712, Val Loss: 0.1735
Epoch 19, Train Loss: 0.1707, Val Loss: 0.1735
Epoch 20, Train Loss: 0.1704, Val Loss: 0.1733
Epoch 21, Train Loss: 0.1700, Val Loss: 0.1729
Epoch 22, Trai

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3809788512.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru

Fold 1 - MAE: 0.4008, RMSE: 0.6762

Fold 2
Epoch 1, Train Loss: 0.1845, Val Loss: 0.1808
Epoch 2, Train Loss: 0.1792, Val Loss: 0.1795
Epoch 3, Train Loss: 0.1782, Val Loss: 0.1791
Epoch 4, Train Loss: 0.1776, Val Loss: 0.1785
Epoch 5, Train Loss: 0.1771, Val Loss: 0.1780
Epoch 6, Train Loss: 0.1765, Val Loss: 0.1773
Epoch 7, Train Loss: 0.1761, Val Loss: 0.1774
Epoch 8, Train Loss: 0.1754, Val Loss: 0.1767
Epoch 9, Train Loss: 0.1750, Val Loss: 0.1758
Epoch 10, Train Loss: 0.1745, Val Loss: 0.1754
Epoch 11, Train Loss: 0.1740, Val Loss: 0.1746
Epoch 12, Train Loss: 0.1734, Val Loss: 0.1745
Epoch 13, Train Loss: 0.1730, Val Loss: 0.1743
Epoch 14, Train Loss: 0.1726, Val Loss: 0.1738
Epoch 15, Train Loss: 0.1720, Val Loss: 0.1736
Epoch 16, Train Loss: 0.1717, Val Loss: 0.1731
Epoch 17, Train Loss: 0.1713, Val Loss: 0.1729
Epoch 18, Train Loss: 0.1709, Val Loss: 0.1729
Epoch 19, Train Loss: 0.1708, Val Loss: 0.1727
Epoch 20, Train Loss: 0.1705, Val Loss: 0.1723
Epoch 21, Train Loss: 0.17

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3809788512.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru

Fold 2 - MAE: 0.3997, RMSE: 0.6880

Fold 3
Epoch 1, Train Loss: 0.1842, Val Loss: 0.1831
Epoch 2, Train Loss: 0.1789, Val Loss: 0.1824
Epoch 3, Train Loss: 0.1779, Val Loss: 0.1811
Epoch 4, Train Loss: 0.1772, Val Loss: 0.1803
Epoch 5, Train Loss: 0.1766, Val Loss: 0.1802
Epoch 6, Train Loss: 0.1762, Val Loss: 0.1794
Epoch 7, Train Loss: 0.1757, Val Loss: 0.1789
Epoch 8, Train Loss: 0.1751, Val Loss: 0.1781
Epoch 9, Train Loss: 0.1745, Val Loss: 0.1781
Epoch 10, Train Loss: 0.1740, Val Loss: 0.1773
Epoch 11, Train Loss: 0.1735, Val Loss: 0.1767
Epoch 12, Train Loss: 0.1731, Val Loss: 0.1770
Epoch 13, Train Loss: 0.1727, Val Loss: 0.1762
Epoch 14, Train Loss: 0.1722, Val Loss: 0.1768
Epoch 15, Train Loss: 0.1718, Val Loss: 0.1756
Epoch 16, Train Loss: 0.1714, Val Loss: 0.1754
Epoch 17, Train Loss: 0.1711, Val Loss: 0.1751
Epoch 18, Train Loss: 0.1705, Val Loss: 0.1753
Epoch 19, Train Loss: 0.1701, Val Loss: 0.1749
Epoch 20, Train Loss: 0.1701, Val Loss: 0.1746
Epoch 21, Train Loss: 0.16

...

Fold 3 - MAE: 0.4038, RMSE: 0.6762

Fold 4
Epoch 1, Train Loss: 0.1856, Val Loss: 0.1766
Epoch 2, Train Loss: 0.1803, Val Loss: 0.1757
Epoch 3, Train Loss: 0.1797, Val Loss: 0.1754
Epoch 4, Train Loss: 0.1788, Val Loss: 0.1747
Epoch 5, Train Loss: 0.1782, Val Loss: 0.1750
Epoch 6, Train Loss: 0.1778, Val Loss: 0.1737
Epoch 7, Train Loss: 0.1771, Val Loss: 0.1729
Epoch 8, Train Loss: 0.1765, Val Loss: 0.1724
Epoch 9, Train Loss: 0.1759, Val Loss: 0.1717
Epoch 10, Train Loss: 0.1754, Val Loss: 0.1715
Epoch 11, Train Loss: 0.1749, Val Loss: 0.1712
Epoch 12, Train Loss: 0.1746, Val Loss: 0.1714
Epoch 13, Train Loss: 0.1742, Val Loss: 0.1712
Epoch 14, Train Loss: 0.1735, Val Loss: 0.1705
Epoch 15, Train Loss: 0.1732, Val Loss: 0.1702
Epoch 16, Train Loss: 0.1728, Val Loss: 0.1701
Epoch 17, Train Loss: 0.1725, Val Loss: 0.1697
Epoch 18, Train Loss: 0.1722, Val Loss: 0.1695
Epoch 19, Train Loss: 0.1717, Val Loss: 0.1697
Epoch 20, Train Loss: 0.1715, Val Loss: 0.1693
Epoch 21, Train Loss: 0.17

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3809788512.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru

Fold 4 - MAE: 0.3913, RMSE: 0.6779

Fold 5
Epoch 1, Train Loss: 0.1847, Val Loss: 0.1785
Epoch 2, Train Loss: 0.1798, Val Loss: 0.1779
Epoch 3, Train Loss: 0.1786, Val Loss: 0.1771
Epoch 4, Train Loss: 0.1780, Val Loss: 0.1775
Epoch 5, Train Loss: 0.1775, Val Loss: 0.1759
Epoch 6, Train Loss: 0.1770, Val Loss: 0.1759
Epoch 7, Train Loss: 0.1764, Val Loss: 0.1755
Epoch 8, Train Loss: 0.1758, Val Loss: 0.1745
Epoch 9, Train Loss: 0.1755, Val Loss: 0.1742
Epoch 10, Train Loss: 0.1748, Val Loss: 0.1752
Epoch 11, Train Loss: 0.1744, Val Loss: 0.1738
Epoch 12, Train Loss: 0.1740, Val Loss: 0.1737
Epoch 13, Train Loss: 0.1736, Val Loss: 0.1734
Epoch 14, Train Loss: 0.1731, Val Loss: 0.1732
Epoch 15, Train Loss: 0.1727, Val Loss: 0.1726
Epoch 16, Train Loss: 0.1723, Val Loss: 0.1725
Epoch 17, Train Loss: 0.1721, Val Loss: 0.1724
Epoch 18, Train Loss: 0.1717, Val Loss: 0.1716
Epoch 19, Train Loss: 0.1712, Val Loss: 0.1716
Epoch 20, Train Loss: 0.1711, Val Loss: 0.1717
Epoch 21, Train Loss: 0.17

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/3809788512.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru

Fold 5 - MAE: 0.4012, RMSE: 0.6715

Average MAE: 0.3994
Average RMSE: 0.6780


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# 加载数据
data = pd.read_csv('pivot_50_peak.csv')

# 数据预处理
# 提取特征列和目标列
numerical_features = ['temperature_2m (°C)', 'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_100m (km/h)']
categorical_features = ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos','peak_stage_off_peak','peak_stage_other','peak_stage_peak']
stations = [col for col in data.columns if col.isdigit()] 
# 构造输入X和目标y
X = data[categorical_features + numerical_features].values
y = data[stations].values

# 定义时间窗口
time_window = 24

# 构造时间窗口的数据集
def create_time_window(X, y, time_window):
    X_window, y_window = [], []
    for i in range(len(X) - time_window):
        X_window.append(X[i:i+time_window])
        y_window.append(y[i+time_window])  # 预测当前时间步的目标
    return np.array(X_window), np.array(y_window)

X_window, y_window = create_time_window(X, y, time_window)

# 使用 KFold 进行交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 初始化存储结果的列表
maes = []
rmses = []

# 交叉验证过程
for fold, (train_index, val_index) in enumerate(kf.split(X_window)):
    print(f"\nFold {fold + 1}")

    # 分割数据
    X_train, X_val = X_window[train_index], X_window[val_index]
    y_train, y_val = y_window[train_index], y_window[val_index]

    # 转换为Tensor
    X_train, X_val = map(torch.tensor, (X_train, X_val))
    y_train, y_val = map(torch.tensor, (y_train, y_val))

    # 数据集类
    class TimeSeriesDataset(Dataset):
        def __init__(self, X, y):
            self.X = X
            self.y = y

        def __len__(self):
            return len(self.X)

        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

    # 数据加载器
    train_dataset = TimeSeriesDataset(X_train, y_train)
    val_dataset = TimeSeriesDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # GRU 和 CNN 模型定义
    class GRUModelWithCNN(nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, filters, kernel_size):
            super(GRUModelWithCNN, self).__init__()
            
            # 第一层 CNN
            self.conv1d_1 = nn.Conv1d(
                in_channels=input_dim,       # 输入通道数
                out_channels=filters,        # 可调整的 filter 数
                kernel_size=kernel_size,     # 卷积核大小
                padding=kernel_size // 2     # 保持输出的时间维度不变
            )
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(dropout)
            
            # GRU 层
            self.gru = nn.GRU(
                input_size=filters ,      # GRU 输入维度为第二层 CNN 的输出通道数
                hidden_size=hidden_dim,      # GRU 隐藏单元数
                num_layers=num_layers,       # GRU 层数
                batch_first=True,
                dropout=dropout
            )
            
            # 全连接层
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            # CNN expects input in shape [batch_size, channels, time_steps]
            x = x.transpose(1, 2)  # [batch_size, time_steps, input_dim] -> [batch_size, input_dim, time_steps]
            
            # 第一层 CNN
            x = self.conv1d_1(x)
            x = self.relu(x)
            x = self.dropout(x)  # Apply dropout after activation
            
            # 调整回 GRU 输入格式
            x = x.transpose(1, 2)  # [batch_size, filters, time_steps] -> [batch_size, time_steps, filters]
            
            # GRU 层
            out, _ = self.gru(x)
            
            # 取 GRU 最后一个时间步的输出
            out = self.fc(out[:, -1, :])
            return out

    # 模型初始化
    input_dim = X_train.shape[2]  # 输入特征维度
    hidden_dim = 64               # GRU 隐藏单元数
    output_dim = y_train.shape[1] # 输出维度
    num_layers = 2                # GRU 层数
    dropout = 0.25                # Dropout 概率

    # 调整 filter 和卷积核大小
    filters = 64               # CNN 的 filter 数量
    kernel_size = 3              # 卷积核大小

    model = GRUModelWithCNN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        num_layers=num_layers,
        dropout=dropout,
        filters=filters,
        kernel_size=kernel_size
    )

    # 打印模型结构（可选）
    print(model)

    # 使用 Huber Loss 替代 MSELoss，并设置 delta 参数
    criterion = HuberLoss(delta=1.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # 训练模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_val_loss = float('inf')
    patience = 5
    trigger_times = 0

    for epoch in range(100):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.float().to(device), y_batch.float().to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)  # 使用 Huber Loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.float().to(device), y_batch.float().to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)  # 使用 Huber Loss
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # 早停机制
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trigger_times = 0
            torch.save(model.state_dict(), f'best_gru_model_fold{fold+1}.pth')  # 保存最佳模型
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping!")
                break

    # 加载最佳模型
    model.load_state_dict(torch.load(f'best_gru_model_fold{fold+1}.pth'))

    # 测试模型
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.float().to(device)
            outputs = model(X_batch)
            y_pred.append(outputs.cpu().numpy())
            y_true.append(y_batch.numpy())

    y_pred = np.vstack(y_pred)
    y_true = np.vstack(y_true)

    # 评估指标
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    maes.append(mae)
    rmses.append(rmse)

    print(f"Fold {fold+1} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# 输出所有折的平均性能
print(f"\nAverage MAE: {np.mean(maes):.4f}")
print(f"Average RMSE: {np.mean(rmses):.4f}")


Fold 1
GRUModelWithCNN(
  (conv1d_1): Conv1d(13, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
  (gru): GRU(64, 64, num_layers=2, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=64, out_features=50, bias=True)
)
Epoch 1, Train Loss: 0.1854, Val Loss: 0.1813
Epoch 2, Train Loss: 0.1797, Val Loss: 0.1799
Epoch 3, Train Loss: 0.1786, Val Loss: 0.1798
Epoch 4, Train Loss: 0.1779, Val Loss: 0.1790
Epoch 5, Train Loss: 0.1777, Val Loss: 0.1790
Epoch 6, Train Loss: 0.1772, Val Loss: 0.1780
Epoch 7, Train Loss: 0.1765, Val Loss: 0.1781
Epoch 8, Train Loss: 0.1761, Val Loss: 0.1776
Epoch 9, Train Loss: 0.1756, Val Loss: 0.1768
Epoch 10, Train Loss: 0.1754, Val Loss: 0.1763
Epoch 11, Train Loss: 0.1749, Val Loss: 0.1765
Epoch 12, Train Loss: 0.1744, Val Loss: 0.1753
Epoch 13, Train Loss: 0.1740, Val Loss: 0.1754
Epoch 14, Train Loss: 0.1739, Val Loss: 0.1750
Epoch 15, Train Loss: 0.1733, Val Loss: 0.1751
Epoch 16, Train 

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/926633453.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru_

Fold 1 - MAE: 0.4006, RMSE: 0.6775

Fold 2
GRUModelWithCNN(
  (conv1d_1): Conv1d(13, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
  (gru): GRU(64, 64, num_layers=2, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=64, out_features=50, bias=True)
)
Epoch 1, Train Loss: 0.1845, Val Loss: 0.1820
Epoch 2, Train Loss: 0.1796, Val Loss: 0.1799
Epoch 3, Train Loss: 0.1786, Val Loss: 0.1788
Epoch 4, Train Loss: 0.1781, Val Loss: 0.1792
Epoch 5, Train Loss: 0.1776, Val Loss: 0.1795
Epoch 6, Train Loss: 0.1771, Val Loss: 0.1779
Epoch 7, Train Loss: 0.1766, Val Loss: 0.1775
Epoch 8, Train Loss: 0.1762, Val Loss: 0.1765
Epoch 9, Train Loss: 0.1758, Val Loss: 0.1763
Epoch 10, Train Loss: 0.1753, Val Loss: 0.1762
Epoch 11, Train Loss: 0.1748, Val Loss: 0.1755
Epoch 12, Train Loss: 0.1743, Val Loss: 0.1751
Epoch 13, Train Loss: 0.1739, Val Loss: 0.1748
Epoch 14, Train Loss: 0.1736, Val Loss: 0.1749
Epoch 15, Train Loss: 0.1735

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/926633453.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru_

Fold 2 - MAE: 0.3977, RMSE: 0.6922

Fold 3
GRUModelWithCNN(
  (conv1d_1): Conv1d(13, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
  (gru): GRU(64, 64, num_layers=2, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=64, out_features=50, bias=True)
)
Epoch 1, Train Loss: 0.1838, Val Loss: 0.1829
Epoch 2, Train Loss: 0.1790, Val Loss: 0.1812
Epoch 3, Train Loss: 0.1782, Val Loss: 0.1812
Epoch 4, Train Loss: 0.1775, Val Loss: 0.1809
Epoch 5, Train Loss: 0.1770, Val Loss: 0.1805
Epoch 6, Train Loss: 0.1764, Val Loss: 0.1796
Epoch 7, Train Loss: 0.1760, Val Loss: 0.1789
Epoch 8, Train Loss: 0.1757, Val Loss: 0.1801
Epoch 9, Train Loss: 0.1754, Val Loss: 0.1778
Epoch 10, Train Loss: 0.1749, Val Loss: 0.1787
Epoch 11, Train Loss: 0.1743, Val Loss: 0.1769
Epoch 12, Train Loss: 0.1741, Val Loss: 0.1777
Epoch 13, Train Loss: 0.1736, Val Loss: 0.1765
Epoch 14, Train Loss: 0.1731, Val Loss: 0.1767
Epoch 15, Train Loss: 0.1729

...

Fold 3 - MAE: 0.4057, RMSE: 0.6775

Fold 4
GRUModelWithCNN(
  (conv1d_1): Conv1d(13, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
  (gru): GRU(64, 64, num_layers=2, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=64, out_features=50, bias=True)
)
Epoch 1, Train Loss: 0.1863, Val Loss: 0.1775
Epoch 2, Train Loss: 0.1809, Val Loss: 0.1765
Epoch 3, Train Loss: 0.1796, Val Loss: 0.1749
Epoch 4, Train Loss: 0.1791, Val Loss: 0.1753
Epoch 5, Train Loss: 0.1785, Val Loss: 0.1740
Epoch 6, Train Loss: 0.1781, Val Loss: 0.1740
Epoch 7, Train Loss: 0.1777, Val Loss: 0.1733
Epoch 8, Train Loss: 0.1771, Val Loss: 0.1727
Epoch 9, Train Loss: 0.1768, Val Loss: 0.1723
Epoch 10, Train Loss: 0.1764, Val Loss: 0.1726
Epoch 11, Train Loss: 0.1760, Val Loss: 0.1716
Epoch 12, Train Loss: 0.1757, Val Loss: 0.1715
Epoch 13, Train Loss: 0.1754, Val Loss: 0.1710
Epoch 14, Train Loss: 0.1751, Val Loss: 0.1710
Epoch 15, Train Loss: 0.1746

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/926633453.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru_

Fold 4 - MAE: 0.3898, RMSE: 0.6784

Fold 5
GRUModelWithCNN(
  (conv1d_1): Conv1d(13, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
  (gru): GRU(64, 64, num_layers=2, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=64, out_features=50, bias=True)
)
Epoch 1, Train Loss: 0.1860, Val Loss: 0.1789
Epoch 2, Train Loss: 0.1802, Val Loss: 0.1788
Epoch 3, Train Loss: 0.1791, Val Loss: 0.1770
Epoch 4, Train Loss: 0.1785, Val Loss: 0.1769
Epoch 5, Train Loss: 0.1779, Val Loss: 0.1764
Epoch 6, Train Loss: 0.1774, Val Loss: 0.1758
Epoch 7, Train Loss: 0.1767, Val Loss: 0.1754
Epoch 8, Train Loss: 0.1765, Val Loss: 0.1752
Epoch 9, Train Loss: 0.1760, Val Loss: 0.1754
Epoch 10, Train Loss: 0.1756, Val Loss: 0.1741
Epoch 11, Train Loss: 0.1752, Val Loss: 0.1743
Epoch 12, Train Loss: 0.1748, Val Loss: 0.1743
Epoch 13, Train Loss: 0.1745, Val Loss: 0.1734
Epoch 14, Train Loss: 0.1741, Val Loss: 0.1731
Epoch 15, Train Loss: 0.1737

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_96800/926633453.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_gru_

Fold 5 - MAE: 0.3974, RMSE: 0.6743

Average MAE: 0.3982
Average RMSE: 0.6800
